## Tensorflow MNIST Dataset

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


## Learning Parameters

In [14]:
import tensorflow as tf 

# parameters 
save_model_file = './train_model.ckpt'
learning_rate = .001
training_epochs = 200
batch_size = 128
display_step = 1

n_input = 784 # 28*28 images
n_classes = 10 # 0-9 digits

# Hidden layer parameter determines size of hidden layer (width of the layer)
n_hidden_layer = 256

In [8]:
import numpy as np
# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

In [9]:
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}

biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# graph input 
x = tf.placeholder("float", [None, 28, 28, 1])
y = tf.placeholder("float", [None, n_classes])

x_flattened = tf.reshape(x, [-1, n_input])

## Multilayer Perceptron

![](image/multi-layer.png)

In [10]:
Z1 = tf.add(tf.matmul(x_flattened, weights['hidden_layer']), biases['hidden_layer'])
A1 = tf.nn.relu(Z1)
Z2 = tf.add(tf.matmul(Z1, weights['out']), biases['out'])

## Optimizer

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z2, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [12]:
# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(Z2, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={x: last_features, y: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={x: valid_features, y: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

## Session 

In [15]:
init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        batches = mnist.train.num_examples//batch_size
        
        batch_x = batch_y = None
        for batch in range(batches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        
        if epoch % 10 == 0:
            print_epoch_stats(epoch, sess, batch_x, batch_y)
            
    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={x: test_features, y: test_labels})
    
    print('Test Accuracy: {}'.format(test_accuracy))
    saver.save(sess, save_model_file)
    print('Trained Model Saved.')

Epoch: 0    - Cost: 41.4     Valid Accuracy: 0.531
Epoch: 10   - Cost: 13.2     Valid Accuracy: 0.828
Epoch: 20   - Cost: 6.84     Valid Accuracy: 0.849
Epoch: 30   - Cost: 4.65     Valid Accuracy: 0.863
Epoch: 40   - Cost: 8.21     Valid Accuracy: 0.869
Epoch: 50   - Cost: 6.19     Valid Accuracy: 0.872
Epoch: 60   - Cost: 4.54     Valid Accuracy: 0.876
Epoch: 70   - Cost: 1.4      Valid Accuracy: 0.877
Epoch: 80   - Cost: 1.99     Valid Accuracy: 0.878
Epoch: 90   - Cost: 2.65     Valid Accuracy: 0.88 
Epoch: 100  - Cost: 4.01     Valid Accuracy: 0.882
Epoch: 110  - Cost: 5.47     Valid Accuracy: 0.882
Epoch: 120  - Cost: 3.82     Valid Accuracy: 0.881
Epoch: 130  - Cost: 4.66     Valid Accuracy: 0.881
Epoch: 140  - Cost: 3.35     Valid Accuracy: 0.882
Epoch: 150  - Cost: 4.79     Valid Accuracy: 0.884
Epoch: 160  - Cost: 1.57     Valid Accuracy: 0.884
Epoch: 170  - Cost: 2.98     Valid Accuracy: 0.884
Epoch: 180  - Cost: 3.03     Valid Accuracy: 0.884
Epoch: 190  - Cost: 4.29     Va

# Save and Restore TF Models 

TF allows you to save using `tf.train.Saver`, saves any `tf.Variable` to your file system

If you're using TensorFlow 0.11.0RC1 or newer, a file called "model.ckpt.meta" will also be created. This file contains the TensorFlow graph.



In [3]:
save_file = './model.ckpt'

weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))
    
    saver.save(sess, save_file)

Weights:
[[-0.80411756  0.61981     1.2196643 ]
 [-0.9379029  -0.5828782   0.19149642]]
Bias:
[-0.6835406  1.6459956  0.9417124]


## Loading Variables

In [4]:
tf.reset_default_graph()

weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)
    
    print('Weight:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

INFO:tensorflow:Restoring parameters from ./model.ckpt
Weight:
[[-0.80411756  0.61981     1.2196643 ]
 [-0.9379029  -0.5828782   0.19149642]]
Bias:
[-0.6835406  1.6459956  0.9417124]


## Loading Saved MultiLayer Perceptron

In [16]:
saver = tf.train.Saver()

with tf.Session() as sess: 
    saver.restore(sess, save_file)
    test_accuracy = sess.run(
        accuracy,
        feed_dict={x: mnist.test.images, y: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))
    

INFO:tensorflow:Restoring parameters from ./train_model.ckpt
Test Accuracy: 0.8851000070571899
